In [ ]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

# Cargo los datos

In [ ]:
prop = pickle.load(open("../../../datos/Data.p","rb"))

In [ ]:
proper = pickle.load(open("../../../datos/propiedadesPruebaConColTransGas.p", "rb"))

propTOT = pd.merge(proper, prop, how="outer")

propTOT['surface_total_in_m2'] = propTOT.apply(lambda row: row[12] if(np.all(pd.isnull(row[11]))) else row[11], axis=1)

propTOT = propTOT[(propTOT['price_usd_per_m2']<8000)]

In [ ]:
#se formatea lat y lon
p = propTOT[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]
p.dropna(inplace = True)

p['lat'] = p.apply(lambda row: row[4][0], axis=1)

p['lon'] = p.apply(lambda row: row[4][1], axis=1)

p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

In [ ]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [ ]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

In [ ]:
def asignarPV(name, dic):
    return dic[name]   

In [ ]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [ ]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [ ]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [ ]:
pEnt = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

In [ ]:
pEnt.dropna(inplace=True)

In [ ]:
proper = pickle.load(open("../../../datos/propiedadesPruebaConColTransGas.p", "rb"))

#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'price_aprox_usd']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

#completo las superficies totales con las cubiertas
p['surface_total_in_m2'] = p.apply(lambda row: row[3] if(np.all(pd.isnull(row[2]))) else row[2], axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

In [ ]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(pEnt[['property_type','surface_total_in_m2',
        'place_value']],pEnt[['price_aprox_usd']],test_size=0.0)

In [ ]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model1 = SVC()
estimators.append(('rfr',model1))

model2 = LogisticRegression()
estimators.append(('gbr',model2))

"""
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))"""

# create the ensemble model
ensemble = VotingClassifier(estimators)

In [ ]:
ensemble.fit(Xtrn, Ytrn['price_aprox_usd'])

In [ ]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(Xtrn)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value']])

In [ ]:
pPru['price_usd'] = ensemble.predict(pPru_imp)